# # Preprocesamiento: Lematización con NLTK

**Objetivo:** Integrar la lematización usando NLTK al pipeline de preprocesamiento.


In [4]:
# !pip install nltk

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.datasets import fetch_20newsgroups

In [ ]:
# importamos las funciones necesarias usadas en el notebook anterior
from src.functions import batch_generator, preprocess_text_step1

In [9]:
# Mapeo de etiquetas POS
def get_wordnet_pos(treebank_tag):
    """Convierte etiquetas POS de Penn Treebank a formato WordNet."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# POS Tagging
Es una tecnica comun utilizada para realizar la clasificacion de palabras en una texto (corpus), basado en las definiciones de las palabras y su contexto.

Estamos usando un etiquetador `nltk.pos_tag` que usa un conjunto de etiquetas `Penn Treebank Tagset` el cual este es muy detallado, por ejemplo :

- NN : Sustantivo singular
- NNS : Sustantivo plural
- VBG : Verbo gerundio
- VBD : Verbo pasado, etc ...

El lematizador `WordNetLemmatizer()` de NLTK espera un conjunto de etiquetas muy simple, basado solo en categorias principales:
 
- Sustantivo
- Verbo
- Adjetivo
- Adverbio

In [10]:
# Inicializamos el lematizador de WordNet
lemmatizer = WordNetLemmatizer()

In [11]:
# Cargamos el dataset de 20 Newsgroups
newsgroups_data = fetch_20newsgroups(subset='all',
                                     remove=('headers', 'footers', 'quotes'),
                                     data_home='./data/20newsgroups_cache')
documents = newsgroups_data.data
num_docs = len(documents)
print(f"Número total de documentos: {num_docs}")

Número total de documentos: 18846


In [ ]:
def preprocess_and_lemmatize_nltk(text_chunk):
    """
    Pipeline completo: Normaliza, tokeniza, etiqueta POS y lematiza con NLTK.
    """
    # normaliza y convierte a minúsculas
    normalized_lower_chunk = preprocess_text_step1(text_chunk)
    # tokeniza usando ingles
    tokens = nltk.word_tokenize(normalized_lower_chunk, language='english')
    # realizamos el etiqueado pos
    pos_tags = nltk.pos_tag(tokens)
    # lematizamos
    lemmas = []
    for word, tag in pos_tags:
        # filtra tokens no alfanuméricos
        if word.isalnum():
            wordnet_tag = get_wordnet_pos(tag)
            lemma = lemmatizer.lemmatize(word, pos=wordnet_tag)
            lemmas.append(lemma)

    return lemmas

*NO son alfanuméricos:*
- Espacios en blanco (' ')
- Signos de puntuación (., ,, ;, !, ?, -, etc.)
- Símbolos ($, %, &, #, @, etc.)
- Caracteres de control (como saltos de línea \n o tabulaciones \t)

# Prueba del Pipeline en un lote

In [ ]:
# obtenemos un lote de prueba
batch_size = 10
batch_gen = batch_generator(documents, batch_size)
first_batch = next(batch_gen, [])
processed_lemmas_batch = [preprocess_and_lemmatize_nltk(doc) for doc in first_batch]

In [ ]:
num_examples_to_show = 3
print(f"\nMostrando los primeros {num_examples_to_show} ejemplos procesados del lote:")

for i in range(min(num_examples_to_show, len(first_batch))):
    print(f"\n--- Ejemplo {i+1} ---")
    original_text = first_batch[i]
    processed_lemmas = processed_lemmas_batch[i]
    #  texto original
    print(f"  Original ({len(original_text)} chars):\n'{original_text[:100]}...'")
    #  texto normalizado/minúsculas
    normalize_text = preprocess_text_step1(original_text)
    print(f"\n  Normalizado/Minúsculas ({len(normalize_text)} chars):\n'{normalize_text[:100]}...'")
    # los lemas resultantes
    print(f"\n  Lemas ({len(processed_lemmas)} tokens):\n{processed_lemmas[:30]}...")



Mostrando los primeros 3 ejemplos procesados del lote:

--- Ejemplo 1 ---
  Original (712 chars):
'

I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about ...'

  Normalizado/Minúsculas (712 chars):
'

i am sure some bashers of pens fans are pretty confused about the lack
of any kind of posts about ...'

  Lemas (136 tokens):
['i', 'be', 'sure', 'some', 'bashers', 'of', 'pen', 'fan', 'be', 'pretty', 'confuse', 'about', 'the', 'lack', 'of', 'any', 'kind', 'of', 'post', 'about', 'the', 'recent', 'pen', 'massacre', 'of', 'the', 'devil', 'actually', 'i', 'be']...

--- Ejemplo 2 ---
  Original (324 chars):
'My brother is in the market for a high-performance video card that supports
VESA local bus with 1-2M...'

  Normalizado/Minúsculas (324 chars):
'my brother is in the market for a high-performance video card that supports
vesa local bus with 1-2m...'

  Lemas (44 tokens):
['my', 'brother', 'be', 'in', 'the', 'market', 'for', 'a', 'video', 'card